In [1]:
import mne
import numpy as np
from pathlib import Path
import time

from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from ica_benchmark.scoring import SCORING_FN_DICT, apply_pairwise_parallel
from ica_benchmark.processing.ica import get_all_methods, get_ica_instance

from ica_benchmark.io.load import BCI_IV_Comp_Dataset
from sklearn.cross_decomposition import PLSCanonical

from sacred.observers import MongoObserver, FileStorageObserver
from sacred import Experiment

import json

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV

from mne.decoding import CSP
from mne.time_frequency import psd_multitaper, psd_welch

def cue_name(cue):
    return {
        0: "Left hand",
        1: "Right hand",
        2: "Foot",
        3: "Tongue",
    }[cue]

class ConcatenateChannelsPSD(BaseEstimator):
    def __init__(self):
        super(ConcatenateChannelsPSD).__init__()

    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        n = len(x)
        return x.reshape(n, -1)


class PSD(BaseEstimator):
    BANDS_DICT = {
#         "delta": (1, 4),
#         "theta": (4, 8),
#         "mu": (8, 13),
        "mu": (8, 12),
#         "beta": (13, 25),
        "beta": (18, 25),
#         "gamma": (25, 40)
    }
    def __init__(self, **kwargs):
        super(PSD).__init__()
        self.kwargs = kwargs
        
    def set_params(**params):
        for param in params:
            assert params in ["picks", "n_fft", "n_overlap", "n_per_seg"]
        self.kwargs.update(params)
    
    def get_params(self, *args, **kwargs):
        return self.kwargs
        
    def fit(self, x, y=None):
        return self

    def transform(self, x, y=None):
        if isinstance(x, list):
            x = mne.concatenate_epochs(x)
        psds, freqs = psd_welch(x, ** self.kwargs)
        if ("average" in self.kwargs) and (self.kwargs["average"] is None):
            psds = psds.sum(axis=3)
        self.freqs = freqs

        band_spectras = list()
        for band, (lfreq, hfreq) in self.BANDS_DICT.items():
            band_spectra = psds[:, :, (freqs >= lfreq) & (freqs < hfreq)]
            band_spectras.append(
                band_spectra.sum(axis=2, keepdims=True)
            )
        
        band_spectras = np.concatenate(band_spectras, axis=2)
            
        return band_spectras


root = Path("/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/")
selected_channels = ["EEG-Fz", "EEG-C3", "EEG-C4", "EEG-Pz", "EEG-Cz"]
filepaths = sorted(root.glob("A*T.gdf"))

# clf.predict(full_epochs)

In [2]:
# psd_welch(train_epochs, n_fft=125, n_overlap=70, n_per_seg=125, average=None)[0].shape

In [3]:
import functools

TEST_LABEL_FOLDER = Path("/home/paulo/Documents/datasets/BCI_Comp_IV_2a/true_labels/")
DEFAULT_TIME_BANDS = [(3, 6)]
def preprocess_epochs(epochs, timebands=None, car_filter=True):
    if timebands is None:
        timebands = DEFAULT_TIME_BANDS
    epochs = epochs.copy()
    
    epochs.load_data()
#     epochs.filter(5, 60)
    
    if car_filter:
        epochs.set_eeg_reference("average")
    events = epochs.events[:, 2]
#     epochs.drop(~((events == 0) | (events == 1)))

    partial_epochs = list()
#     for timeband in [(3, 5), (4, 6)]:
#     for timeband in [(0, 2), (1, 3), (3, 5), (4, 6)]:
#     for timeband in [(3, 4), (4, 5), (5, 6)]:
#     for timeband in [(3, 6)]:
#         partial_epochs.append(
#             epochs.copy().crop(*timeband).shift_time(-timeband[0])
#         )
#     epochs = mne.concatenate_epochs(partial_epochs).copy()
    return epochs

def load_subject_epochs(subject_number, test_label_folder=TEST_LABEL_FOLDER):
    train_file_path = root / "A{}T.gdf".format(str(subject_number).rjust(2, "0"))
    test_file_path = root / "A{}E.gdf".format(str(subject_number).rjust(2, "0"))
    test_label_file_path = test_label_folder / "A{}E.csv".format(str(subject_number).rjust(2, "0"))
    train_epochs = BCI_IV_Comp_Dataset.load_dataset(
        [train_file_path],
        reject=False,
        as_epochs=True,
        concatenate=False,
        drop_bad=False,
        return_metadata=False,
        tmin=0.,
        tmax=5.5,
    )[0]
    test_epochs = BCI_IV_Comp_Dataset.load_dataset(
        [test_file_path],
        reject=False,
        as_epochs=True,
        concatenate=False,
        drop_bad=False,
        return_metadata=False,
        tmin=0.,
        # The last timestamp does not exist, so MNE will ignore the last epoch because it will not end in 6s
        # So here we use 5.5 seconds because there will always be 5.5 seconds after a event
        tmax=5.5,
        has_labels=False
    )[0]

    train_epochs = preprocess_epochs(train_epochs, car_filter=False)
    test_epochs = preprocess_epochs(test_epochs, car_filter=False)
    
#     train_epochs.drop_bad(dict(eeg=1e-4))

    train_events = train_epochs.events[:, 2].flatten()
    train_epochs.drop(~((train_events == 0) | (train_events == 1)))
    train_labels = train_epochs.events[:, 2]
    
    test_labels = pd.read_csv(test_label_file_path, header=None).to_numpy().flatten() - 1
    assert len(test_labels) == len(test_epochs), "{} epochs | {} labels".format(len(test_epochs), len(test_labels))
    
    test_epochs.events[:, 2] = test_labels
    test_epochs.drop(~((test_labels == 0) | (test_labels == 1)))
    test_labels = test_epochs.events[:, 2]

    return (train_epochs, train_labels), (test_epochs, test_labels)



In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import make_scorer
from sklearn.feature_selection import SequentialFeatureSelector


def get_corr_features(x):
    features = list()
    for i in range(len(x)):
        corr_matrix = np.corrcoef(x[i])
        idx1, idx2 = np.triu_indices(corr_matrix.shape[0], 1)
        features.append(
            corr_matrix[idx1, idx2].reshape(1, -1)
        )
    return np.vstack(features)


class CSPWrapper(BaseEstimator):
    def __init__(self, n_components=4):
        self.n_components = n_components
        self.csp = None

    def fit(self, x, y):
        self.csp = CSP(n_components=self.n_components)
        self.csp.fit(x, y)
        return self
    
    def transform(self, x):
        return self.csp.transform(x)
    
    def set_params(self, **params):
        self.n_components = params["n_components"]
        self.csp = CSP(n_components=self.n_components)
        return self
    
    def get_params(self, deep=True):
        return dict(n_components=self.n_components)
    
def run(use_ica=True, use_csp=False, channels=None):
    global train_epochs, train_labels, test_epochs, test_labels

    scores_dict = dict(bas=list(), kappa=list(), acc=list())

    for filepath in filepaths:
        print(filepath)
        subject_number = int(filepath.name[1:3])
        (train_epochs, train_labels), (test_epochs, test_labels) = load_subject_epochs(subject_number)
        
#         n_train = 3 * len(train_epochs) // 4
#         test_epochs = train_epochs[n_train:]
#         test_labels = train_labels[n_train:]
#         train_epochs = train_epochs[:n_train]
#         train_labels = train_labels[:n_train]

        selected_channels = channels
        if channels is None:
            selected_channels = train_epochs.ch_names
            
        train_epochs.pick(selected_channels)
        test_epochs.pick(selected_channels)
        
        ICA = get_ica_instance("ext_infomax")
        ica_channels = ["ICA{}".format(str(i).rjust(3, "0")) for i in range(len(selected_channels))]    

        x_train, y_train = train_epochs, train_labels
        x_test, y_test = test_epochs, test_labels

        if use_ica:
            ICA.fit(x_train)
            x_train = ICA.transform(x_train)
            x_test = ICA.transform(x_test)

        if use_csp:
            x_train = x_train.get_data()
            x_test = x_test.get_data()
            
        print(filepath.name)
        print("Train size", len(x_train))
        print("Test size", len(x_test))
        classifier = LDA(n_components=1)
        SFS = SequentialFeatureSelector(
            LDA(n_components=1),
            direction='forward',
            cv=4,
            scoring=make_scorer(cohen_kappa_score, greater_is_better=True)
        )
        param_grid = dict(
#             svc__C=np.logspace(-2, 2, 10),
#             svc__C=[.01, .1, 1, 10],
#             selectkbest__k=[10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, ]
#             lineardiscriminantanalysis__n_components=[2, 4, 6]
#             csp__n_components=[1, 2, 3, 4, 5]
            sequentialfeatureselector__n_features_to_select=[1, 2, 3, 5, 10, 15, 20, 30],
#             plscanonical__n_components=[1, 2, 3, 4, 5]
        )
        if not use_csp:
            len_size = 250
            psd = PSD(
                picks=x_train.ch_names,
                n_fft=1 * len_size,
                n_overlap=len_size // 4,
                n_per_seg=1 * len_size,
                average="mean",
                window="hamming",
                proj=False
            )
            psd.fit(x_train)
            x_train = psd.transform(x_train)
            x_test = psd.transform(x_test)

            # 2 classes
            clf = make_pipeline(
                ConcatenateChannelsPSD(),
                StandardScaler(),
                SFS,
                classifier
            )
        else:
            clf = make_pipeline(
                CSPWrapper(),
                StandardScaler(),
                SFS,
                classifier
            )
            param_grid.update(
                cspwrapper__n_components=[1, 2, 3, 4, 5, 10],
                sequentialfeatureselector__n_features_to_select=[1, 2, 3, 4, 5]
            )
            
        gs_cv = GridSearchCV(
            clf,
            param_grid=param_grid,
            cv=4,
            scoring=make_scorer(cohen_kappa_score, greater_is_better=True),
            error_score=-1,
            refit=True,
            n_jobs=3,
            verbose=2
        )
        print("Fitting... ", end="")
        start = time.time()
        gs_cv.fit(x_train, y_train)
        print("Done (took {:.2f}s)".format(time.time() - start))

        pred  = gs_cv.predict(x_test)
        bas = balanced_accuracy_score(y_test, pred)
        acc = accuracy_score(y_test, pred)
        kappa = cohen_kappa_score(y_test, pred)

        scores_dict["kappa"].append(kappa)
        scores_dict["acc"].append(acc)
        scores_dict["bas"].append(bas)

        print("Kappa", kappa)
        print("Accuracy", acc)
        print("BAS", bas)
        print(gs_cv.best_score_)
        print(gs_cv.best_params_)
        print(classification_report(y_test, pred), end="\n\n")
        

    print(
        "Avg Kappa: {:.3f} (std. {:.3f})".format(
            np.mean(scores_dict["kappa"]),
            np.std(scores_dict["kappa"]),
        )
    )


In [5]:
run(use_ica=False, use_csp=False)

/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf
A01T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits
Done (took 95.51s)
Kappa 0.13888888888888884
Accuracy 0.5694444444444444
BAS 0.5694444444444444
0.25
{'sequentialfeatureselector__n_features_to_select': 20}
              precision    recall  f1-score   support

           0       0.58      0.53      0.55        72
           1       0.56      0.61      0.59        72

    accuracy                           0.57       144
   macro avg       0.57      0.57      0.57       144
weighted avg       0.57      0.57      0.57       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A02T.gdf
A02T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits
Done (took 102.17s)
Kappa 0.04166666666666663
Accuracy 0.5208333333333334
BAS 0.5208333333333334
0.2361111111111111
{'sequentialfeatureselector__n_features_to_select': 30}
       

In [6]:
run(use_ica=True, use_csp=False)

/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf
A01T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits
Done (took 158.17s)
Kappa 0.25
Accuracy 0.625
BAS 0.625
0.29166666666666663
{'sequentialfeatureselector__n_features_to_select': 30}
              precision    recall  f1-score   support

           0       0.60      0.74      0.66        72
           1       0.66      0.51      0.58        72

    accuracy                           0.62       144
   macro avg       0.63      0.62      0.62       144
weighted avg       0.63      0.62      0.62       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A02T.gdf
A02T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits
Done (took 153.44s)
Kappa -0.01388888888888884
Accuracy 0.4930555555555556
BAS 0.4930555555555556
0.15277777777777776
{'sequentialfeatureselector__n_features_to_select': 30}
              precision    rec

In [7]:
run(use_ica=True, use_csp=False, channels=["EEG-C3", "EEG-C4"])

/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf
A01T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 32.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transfor

Done (took 1.56s)
Kappa 0.05555555555555558
Accuracy 0.5277777777777778
BAS 0.5277777777777778
0.125
{'sequentialfeatureselector__n_features_to_select': 3}
              precision    recall  f1-score   support

           0       0.55      0.33      0.41        72
           1       0.52      0.72      0.60        72

    accuracy                           0.53       144
   macro avg       0.53      0.53      0.51       144
weighted avg       0.53      0.53      0.51       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A02T.gdf
A02T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 32.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transfor

Done (took 1.41s)
Kappa -0.09722222222222232
Accuracy 0.4513888888888889
BAS 0.4513888888888889
0.1388888888888889
{'sequentialfeatureselector__n_features_to_select': 2}
              precision    recall  f1-score   support

           0       0.46      0.53      0.49        72
           1       0.44      0.38      0.41        72

    accuracy                           0.45       144
   macro avg       0.45      0.45      0.45       144
weighted avg       0.45      0.45      0.45       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A03T.gdf
A03T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits


...

/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 32.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transfor

Done (took 0.98s)
Kappa 0.18055555555555558
Accuracy 0.5902777777777778
BAS 0.5902777777777778
0.02777777777777779
{'sequentialfeatureselector__n_features_to_select': 3}
              precision    recall  f1-score   support

           0       0.78      0.25      0.38        72
           1       0.55      0.93      0.69        72

    accuracy                           0.59       144
   macro avg       0.67      0.59      0.54       144
weighted avg       0.67      0.59      0.54       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A08T.gdf
A08T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 32.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transfor

Done (took 1.03s)
Kappa 0.25
Accuracy 0.625
BAS 0.625
0.20833333333333337
{'sequentialfeatureselector__n_features_to_select': 3}
              precision    recall  f1-score   support

           0       0.58      0.86      0.70        72
           1       0.74      0.39      0.51        72

    accuracy                           0.62       144
   macro avg       0.66      0.62      0.60       144
weighted avg       0.66      0.62      0.60       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A09T.gdf
A09T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 8 candidates, totalling 32 fits
Done (took 0.92s)
Kappa 0.6388888888888888
Accuracy 0.8194444444444444
BAS 0.8194444444444444
0.23611111111111113
{'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.83      0.81      0.82        72
           1       0.81      0.83      0.82        72

    accuracy                           0.8

/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 32.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transfor

In [8]:
run(use_ica=False, use_csp=True)

/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf
A01T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transf

Done (took 86.62s)
Kappa -0.04166666666666674
Accuracy 0.4791666666666667
BAS 0.47916666666666663
0.15277777777777782
{'cspwrapper__n_components': 5, 'sequentialfeatureselector__n_features_to_select': 3}
              precision    recall  f1-score   support

           0       0.38      0.07      0.12        72
           1       0.49      0.89      0.63        72

    accuracy                           0.48       144
   macro avg       0.44      0.48      0.37       144
weighted avg       0.44      0.48      0.37       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A02T.gdf
A02T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transf

Done (took 79.45s)
Kappa 0.01388888888888884
Accuracy 0.5069444444444444
BAS 0.5069444444444444
0.27777777777777773
{'cspwrapper__n_components': 4, 'sequentialfeatureselector__n_features_to_select': 2}
              precision    recall  f1-score   support

           0       0.50      0.94      0.66        72
           1       0.56      0.07      0.12        72

    accuracy                           0.51       144
   macro avg       0.53      0.51      0.39       144
weighted avg       0.53      0.51      0.39       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A03T.gdf
A03T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


...

Done (took 53.67s)
Kappa 0.23611111111111116
Accuracy 0.6180555555555556
BAS 0.6180555555555556
0.7222222222222223
{'cspwrapper__n_components': 10, 'sequentialfeatureselector__n_features_to_select': 4}
              precision    recall  f1-score   support

           0       0.87      0.28      0.42        72
           1       0.57      0.96      0.72        72

    accuracy                           0.62       144
   macro avg       0.72      0.62      0.57       144
weighted avg       0.72      0.62      0.57       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A08T.gdf
A08T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transf

Done (took 44.59s)
Kappa 0.875
Accuracy 0.9375
BAS 0.9375
0.8611111111111112
{'cspwrapper__n_components': 5, 'sequentialfeatureselector__n_features_to_select': 4}
              precision    recall  f1-score   support

           0       0.97      0.90      0.94        72
           1       0.91      0.97      0.94        72

    accuracy                           0.94       144
   macro avg       0.94      0.94      0.94       144
weighted avg       0.94      0.94      0.94       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A09T.gdf
A09T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transf

Done (took 40.18s)
Kappa -0.02777777777777768
Accuracy 0.4861111111111111
BAS 0.4861111111111111
0.3472222222222222
{'cspwrapper__n_components': 4, 'sequentialfeatureselector__n_features_to_select': 2}
              precision    recall  f1-score   support

           0       0.49      0.81      0.61        72
           1       0.46      0.17      0.24        72

    accuracy                           0.49       144
   macro avg       0.48      0.49      0.43       144
weighted avg       0.48      0.49      0.43       144


Avg Kappa: 0.213 (std. 0.313)


In [9]:
run(use_ica=False, use_csp=True, channels=["EEG-C3", "EEG-C4"])

/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A01T.gdf
A01T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_trans

Done (took 2.58s)
Kappa -0.05555555555555558
Accuracy 0.4722222222222222
BAS 0.4722222222222222
0.04166666666666666
{'cspwrapper__n_components': 2, 'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.46      0.33      0.39        72
           1       0.48      0.61      0.54        72

    accuracy                           0.47       144
   macro avg       0.47      0.47      0.46       144
weighted avg       0.47      0.47      0.46       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A02T.gdf
A02T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_trans

Done (took 2.45s)
Kappa 0.01388888888888884
Accuracy 0.5069444444444444
BAS 0.5069444444444444
0.15277777777777776
{'cspwrapper__n_components': 2, 'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.52      0.15      0.24        72
           1       0.50      0.86      0.64        72

    accuracy                           0.51       144
   macro avg       0.51      0.51      0.44       144
weighted avg       0.51      0.51      0.44       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A03T.gdf
A03T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


...

Done (took 2.57s)
Kappa 0.04166666666666663
Accuracy 0.5208333333333334
BAS 0.5208333333333334
0.08333333333333334
{'cspwrapper__n_components': 2, 'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.52      0.56      0.54        72
           1       0.52      0.49      0.50        72

    accuracy                           0.52       144
   macro avg       0.52      0.52      0.52       144
weighted avg       0.52      0.52      0.52       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A08T.gdf
A08T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_trans

Done (took 2.95s)
Kappa 0.11111111111111116
Accuracy 0.5555555555555556
BAS 0.5555555555555556
0.20833333333333334
{'cspwrapper__n_components': 2, 'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.55      0.61      0.58        72
           1       0.56      0.50      0.53        72

    accuracy                           0.56       144
   macro avg       0.56      0.56      0.55       144
weighted avg       0.56      0.56      0.55       144


/home/paulo/Documents/datasets/BCI_Comp_IV_2a/gdf/A09T.gdf
A09T.gdf
Train size 144
Test size 144
Fitting... Fitting 4 folds for each of 30 candidates, totalling 120 fits


/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to -1.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/paulo/anaconda3/envs/bci/lib/python3.8/site-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_trans

Done (took 3.03s)
Kappa -0.09722222222222232
Accuracy 0.4513888888888889
BAS 0.4513888888888889
0.125
{'cspwrapper__n_components': 2, 'sequentialfeatureselector__n_features_to_select': 1}
              precision    recall  f1-score   support

           0       0.46      0.50      0.48        72
           1       0.45      0.40      0.42        72

    accuracy                           0.45       144
   macro avg       0.45      0.45      0.45       144
weighted avg       0.45      0.45      0.45       144


Avg Kappa: 0.012 (std. 0.060)


In [10]:
(train_epochs, train_labels), (test_epochs, test_labels) = load_subject_epochs(1)
ICA = get_ica_instance("jade")
ica_channels = ["ICA{}".format(str(i).rjust(3, "0")) for i in range(len(selected_channels))]    

x_train, y_train = train_epochs, train_labels
x_test, y_test = test_epochs, test_labels

ICA.fit(x_train)
x_train = ICA.transform(x_train)
x_test = ICA.transform(x_test)
_, _, len_size = x_train.get_data().shape
psd = PSD(
    picks=x_train.ch_names,
    n_fft=len_size // 2,
    n_overlap=len_size // 3,
    n_per_seg=len_size // 2,
    average=None,
    window="hamming"
)
psd.fit(x_train)
x_train = psd.transform(x_train)
x_test = psd.transform(x_test)